# Programação Orientada a Objetos (POO)
## Tema 4
### Parte I – Classes Abstratas
Jaime A. Martins

(CEOT/ISE/UAlg - jamartins@ualg.pt)

###### Autores: Jaime Martins [v2]; Pedro Cardoso [v1]

## Classes abstratas

* Uma classe define as características e o comportamento de um conjunto de objetos.
* No entanto, **nem todas as classes são projetadas para serem instanciadas (permitir a criação de objetos)**
   * Ou seja, algumas classes são usadas apenas para definir um modelo de características comuns e, depois, serem herdadas por outras classes. 

 * As classes que não podem ser instanciadas chamam-se **classes abstratas**:
    * Uma **classe abstrata** corresponde à declaração de uma classe para a qual nunca pretendemos criar objetos/instanciar. 
    * As **classes abstratas** são apenas usadas como **superclasses** em hierarquias de herança. 
    * As **classe abstratas** não podem ser usadas para instanciar objetos, porque são incompletas.

 * As classes que não são abstratas e que podem ser instanciadas, são conhecidas como **classes concretas**:
    * As subclasses devem __implementar as partes ausentes__ para se tornarem classes concretas

As classes abstratas são utilizadas para definir métodos que __têm__ de ser implementados em todas as suas subclasses, sem apresentar uma implementação para os mesmos.
- Esses métodos são chamados de **métodos abstratos**.
- Qualquer classe que possua pelo menos um **método abstrato** é uma **classe abstrata**, mas uma classe pode ser abstrata sem possuir nenhum método abstrato.
- Em algumas linguagens, um **método abstrato** "não tem corpo", ou seja, apresenta-se apenas uma "assinatura".

### Solução 1 (rudimentar)

Declaram-se os métodos e depois levanta-se uma exceção de método não implementado.

In [1]:
class Vehicle:
    def __init__(self, owner, brand):
        self.owner = owner
        self.brand = brand

    def vehicle_info(self):
        raise NotImplementedError("vehicle_info: não implementado")

    @property
    def owner(self):
        return self.__owner

    @owner.setter
    def owner(self, owner):
        self.__owner = owner

    @property
    def brand(self):
        return self.__brand

    @brand.setter
    def brand(self, brand):
        self.__brand = brand

No entanto, a criação de um objeto é valida

In [2]:
carro = Vehicle("Fiat", "Margarida")

Será levantada uma exceção quando se chama o método `vehicle_info()`

In [3]:
carro.vehicle_info()

NotImplementedError: vehicle_info: não implementado

### Solução 2 (a mais correta!)

Como 2ª solução podemos usar o módulo `abc` (_Abstract Base Classes_), prevenindo que a classe possa ser instanciada.

In [4]:
from abc import ABC, abstractmethod


class Vehicle(ABC):  # Herança de ABC
    def __init__(self, owner, brand):
        self.owner = owner
        self.brand = brand

    @abstractmethod
    def vehicle_info(self):
        pass

    @property
    def owner(self):
        return self.__owner

    @owner.setter
    def owner(self, owner):
        self.__owner = owner

    @property
    def brand(self):
        return self.__brand

    @brand.setter
    def brand(self, brand):
        self.__brand = brand

Ou seja, se tentarmos instanciar a classe, será imediatamente levantada uma exceção:

In [5]:
carro = Vehicle("Fiat", "Margarida")

TypeError: Can't instantiate abstract class Vehicle with abstract method vehicle_info

Começemos então por estender a classe `Vehicle` com a classe `Car`

In [6]:
class Car(Vehicle):
    def __init__(self, owner, brand, engine):
        super().__init__(owner, brand)
        self.engine = engine

    @property
    def engine(self):
        return self.__engine

    @engine.setter
    def engine(self, e):
        self.__engine = e

Mas não basta derivar da super classe

In [7]:
carro = Car("Margarida", "Fiat", "1500 turbo")

TypeError: Can't instantiate abstract class Car with abstract method vehicle_info

Temos de implementar os métodos que foram decorados como `@abstractmethod`

In [8]:
class Car(Vehicle):
    def __init__(self, owner, brand, engine):
        super().__init__(owner, brand)
        self.engine = engine

    def vehicle_info(self):  # implementação do método abstrato
        print(self.__dict__)

    @property
    def engine(self):
        return self.__engine

    @engine.setter
    def engine(self, e):
        self.__engine = e

In [9]:
carro = Car("Margarida", "Fiat", "1500 turbo")

In [10]:
carro.vehicle_info()

{'_Vehicle__owner': 'Margarida', '_Vehicle__brand': 'Fiat', '_Car__engine': '1500 turbo'}


## Exemplo

Comecemos por definir uma classe abstrata para jogos de tabuleiro com 2 jogadores que jogam alternadamente

![img-algoritmo_jogo_tabuleiro.png](img/img-algoritmo_jogo_tabuleiro.png)


In [11]:
from abc import ABC, abstractmethod
import random


class Jogo(ABC):
    """
    Classe abstrata que define a interface de um jogo para
    2 jogadores.
    """

    def __init__(self) -> None:
        """
        Inicializar o jogo.
        """
        print("Bom jogo...")
        self.inicializa_tabuleiro()

    @abstractmethod
    def inicializa_tabuleiro(self) -> None:
        """
        Inicializar o tabuleiro do jogo.
        """
        pass

    @abstractmethod
    def mostra_tabuleiro(self) -> None:
        """
        Desenhar o tabuleiro do jogo.
        """
        pass

    @abstractmethod
    def joga_humano(self, jogador: int) -> None:
        """
        Solicitar ao humano :jogador: a próxima jogada
        e colocá-la no tabuleiro.
        :param jogador: número do jogador (0 ou 1).
        """
        pass

    @abstractmethod
    def ha_jogadas_possiveis(self) -> bool:
        """
        Verifica se ainda há jogadas possíveis ou se o jogo
        está empatado.
        :return: `True` se ainda há jogadas possíveis,
        `False` caso contrário.
        """
        pass

    @abstractmethod
    def terminou(self) -> bool:
        """
        Verifica se foi verificada a condição de paragem,
        i.e., um jogador ganhou.
        :return: `True` se o jogo terminou,
        `False` caso contrário.
        """
        pass

    def jogar(self) -> None:
        """
        Corre o jogo.
        """
        # Escolher quem joga em primeiro
        jogador = random.randint(0, 1)

        while True:
            self.mostra_tabuleiro()
            self.joga_humano(jogador)
            if self.terminou():
                self.mostra_tabuleiro()
                print(f"\nO jogador {jogador} ganhou!")
                return
            elif not self.ha_jogadas_possiveis():
                self.mostra_tabuleiro()
                print("\nEmpataram...")
                return
            # Passar a vez ao outro jogador
            jogador = (jogador + 1) % 2  # 0->1 ou 1->0

Agora podemos criar uma classe concreta, definindo somente os métodos abstratos

In [12]:
class Galo(Jogo):
    """
    Classe concreta que herda da classe Jogo e implementa
    o jogo do Galo.
    """

    def inicializa_tabuleiro(self) -> None:
        """
        Inicializa o tabuleiro do jogo do Galo.
        """
        # Conta as jogadas, serve para saber se ainda ha jogadas validas
        self.numero_de_jogadas_realizadas = 0
        # Dicionário que representa o tabuleiro
        self.tabuleiro = {(lin, col): " " for lin in range(3) for col in range(3)}

    def mostra_tabuleiro(self) -> None:
        """
        Desenha o tabuleiro do jogo do Galo.
        """
        print(13 * "-")
        for lin in range(3):
            for col in range(3):
                print(f"| {self.tabuleiro[(lin, col)]} ", end="")
            print("|\n" + 13 * "-")

    def _le_linha_coluna_valida(self, msg) -> int:
        """
        Método auxiliar para ler uma posição que seja 1, 2 ou 3.
        :param s: mensagem para o utilizador
        :return: posição válida lida a partir do input do utilizador
        """
        inputs_validos = {"1", "2", "3"}
        while True:
            pos = input(msg)
            # Verifica se a posição é válida
            if pos in inputs_validos:
                # Devolve int(posição)-1, para ficar entre 0 e 2
                return int(pos) - 1

    def joga_humano(self, jogador) -> None:
        """
        Método que solicita a jogada ao jogador humano
        e a coloca no tabuleiro do jogo do galo.
        :param jogador: número do jogador humano
        """
        print(f"\nJogador {jogador}, insira a sua jogada")
        while True:
            linha = self._le_linha_coluna_valida("Linha? ")
            coluna = self._le_linha_coluna_valida("Coluna? ")

            # Verifica se a posição não esta preenchida, i.e., é valida
            if self.tabuleiro[(linha, coluna)] == " ":
                self.tabuleiro[(linha, coluna)] = ["O", "X"][jogador]
                self.numero_de_jogadas_realizadas += 1
                return
            else:
                print("Jogada inválida. Tente de novo!")

    def terminou(self) -> bool:
        """
        Verifica a condição de paragem, i.e., um jogador ganhou.
        :return: `True` se o jogo terminou, `False` caso contrário.
        """
        posicoes_ganhadoras = (
            ((0, 0), (0, 1), (0, 2)),  # Linha 0
            ((1, 0), (1, 1), (1, 2)),  # Linha 1
            ((2, 0), (2, 1), (2, 2)),  # Linha 2
            ((0, 0), (1, 0), (2, 0)),  # Coluna 0
            ((0, 1), (1, 1), (2, 1)),  # Coluna 1
            ((0, 2), (1, 2), (2, 2)),  # Coluna 2
            ((0, 0), (1, 1), (2, 2)),  # Diagonal
            ((0, 2), (1, 1), (2, 0)),  # Anti-diagonal
        )

        # Devolve True se encontrou posição ganhadora
        return any(
            self.tabuleiro[pos[0]] == self.tabuleiro[pos[1]] == self.tabuleiro[pos[2]]
            and self.tabuleiro[pos[0]] != " "
            for pos in posicoes_ganhadoras
        )

    def ha_jogadas_possiveis(self):
        """
        Verifica se ainda há jogadas possíveis ou se o jogo está
        empatado.
        :return: `True` se ainda há jogadas possíveis,
        `False` caso contrário.
        """
        return self.numero_de_jogadas_realizadas < 9

In [ ]:
jogo = Galo()
jogo.jogar()

Bom jogo...
-------------
|   |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
-------------

Jogador 0, insira a sua jogada
